# Module 8 - ML, XAI, and data analytics

## Explainable AI in Practice

Machine Learning / Deep Learning approaches, such as Neural Networks, are one of the most powerful suporvised techniques in computer science. They were able to contribute to powerful spam filters, image recognition, medical diagnosis, market estimation, etc, that have very high predictive scores.

Despite the rapid advances in machine intelligence, its adoption by human users is impeded by a lack of **trust** in the accuracy of the results generated by the predictive models. Consequently, it is hard for users to gain insights regarding the underlying reasons that have led to certain predictions. 

Without understanding the rationale of the black-box machinery, there will be a lack of trust in the accuracy of the predictions, thus a reluctance to use the predictions, and in the worse case, consequences of an incorrect decision based on the predictions. Moreover, the consequences of misclassifications, either due to biases in data, or simply because the algorithm could not generalise well enough, can be damaging for decision-makers and can put certain societal groups at risk. This is what it is known as the **black box problem**.

<img src="graphics/black_box.png" />


In this lectture, we will be looking at state of the art techniques that try to address the black box problem. Notice that these techniques are not perfect and interpretability and explainability of predictive black boxes is still an open research question and a very hot topic in the litearture.

The algorithm that we will be looking at is LIME (Local Interpretable Model-Agnostic Explanations).


## What is LIME?

LIME (Local Interpretable Model-agnostic Explanations)is a novel explanation technique that explains the prediction of any classifier in an interpretable and faithful manner by learning a interpretable model locally around the prediction.

It works with **ANY CLASSIFIER** and explains a prediction so that even the non experts could compare and improve on an untrustworthy model.

## What is an Explanation?

Intuitively, an explanation is a local linear approximation of the model's behaviour. While the model may be very complex globally, it is easier to approximate it around the vicinity of a particular instance. While treating the model as a black box, we perturb the instance we want to explain and learn a sparse linear model around it, as an explanation. The figure below illustrates the intuition for this procedure. The model's decision function is represented by the blue/pink background, and is clearly nonlinear. The bright red cross is the instance being explained (let's call it X). We sample instances around X, and weight them according to their proximity to X (weight here is indicated by size). We then learn a linear model (dashed line) that approximates the model well in the vicinity of X, but not necessarily globally.

<img src="https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png" />


Source <a href="https://github.com/marcotcr/lime">LIME</a>

In [ ]:
# scatter plot of the circles dataset with points colored by class
from sklearn.datasets import make_circles
from numpy import where
from matplotlib import pyplot

# mlp for the two circles classification problem
from sklearn.datasets import make_circles
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Conv2D, MaxPool2D,  Conv3D, MaxPool3D,Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.initializers import RandomUniform

# keras / deep learning libraries
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

# callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Figure Plotting libraries
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import pyplot
import seaborn as sns
sns.set()

# Machine Learning libraries
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import statistics as stat

# very important for feature transformation
from sklearn.preprocessing import OneHotEncoder, StandardScaler,MinMaxScaler,LabelEncoder

import warnings
warnings.filterwarnings("ignore")


from tensorflow.keras import metrics
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.initializers import glorot_uniform

import lime


## Explaining the Breast Cancer Dataset



In workshop 2B, you had to classify a breast tumour dataset using Naive Bayes algorithm. In this task, you will do classify again the dataset, but using:

- (1) a neural network with the 'tanh' as the activation function (you can choose as many hidden layers as you want)
- (2) a deep neural network with the 'relu' activation function (you can choose as many hidden layers as you want)

In [ ]:
# Let's revisit the breast cancer dataset

file_path = 'data/breast_data_full.csv'
data = pd.read_csv( file_path )
data

In [ ]:
class_var = "diagnosis"
feature_names = data.columns.to_list()
feature_names.remove(class_var)
feature_names.remove( "id")

X = data[feature_names]
y = data[class_var]

# general info about number of features and classes
n_features = X.shape[1]
n_samples = X.shape[0]
n_classes = len(data[class_var].unique())

In [ ]:
# normalize features

enc = OneHotEncoder()
Y = enc.fit_transform(y.values.reshape(-1, 1)).toarray()


In [ ]:
# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:

# separate the dataset into test set, validation set and train set
# YOUR CODE HERE:
#X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3 )
# create the training set and the test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state = 515)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state = 515)

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
callbacks_list = [early_stop]

# define the Machine Learning Model: Neural Network
# YOUR CODE HERE:
model_tum = Sequential()

# add the required layers
# YOUR CODE HERE:
model_tum.add(Dense(10, input_dim= len(feature_names), activation='tanh'))
model_tum.add(Dropout(0.2))
model_tum.add(Dense(7, activation='tanh'))
model_tum.add(Dropout(0.2))
model_tum.add(Dense(3, activation='tanh'))
model_tum.add(Dropout(0.2))
model_tum.add(Dense(2, activation='sigmoid'))

# compile the network using: 
# the 'mean_squared_error' as a loss function
# the stochastic gradient descent ('sgd') as the optimization function
# and the 'accuracy' as evaluation metric
# YOUR CODE HERE:
model_tum.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

# fit the model to the data -> learning the model
# YOUR CODE HERE:
# fit model
history = model_tum.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=300, verbose=0,  callbacks = callbacks_list)


In [ ]:
# evaluate the model
# evaluate the model
train_loss, train_acc = model_tum.evaluate(X_train, y_train, verbose=1)
test_loss, test_acc = model_tum.evaluate(X_test, y_test, verbose=1)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
print('Train: %.3f, Test: %.3f' % (train_loss, test_loss))

In [ ]:
# plot training history
metric = "accuracy"
pyplot.plot(history.history[metric], label='train')
pyplot.plot(history.history["val_" + metric], label='val')
pyplot.ylabel('accuracy', fontsize=12)
pyplot.xlabel('iterations', fontsize=12)
pyplot.legend()
pyplot.show()

In [ ]:
# plot training history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.ylabel('loss', fontsize=12)
pyplot.xlabel('iterations', fontsize=12)
pyplot.legend()
pyplot.show()

## Explaining Predictions using LIME (Local interpretable model-agnostic explanations)

In [ ]:
# let's look at the confusion matrix

from sklearn.metrics import confusion_matrix
from matplotlib import cm

groundtruth = enc.inverse_transform( y_test )
predictions = enc.inverse_transform( model_tum.predict( X_test ) )

mat = confusion_matrix(groundtruth, predictions)
sns.heatmap(mat.T, square=True, cbar=True, xticklabels=["Benign, Malignant"], \
            yticklabels=["Benign, Malignant"], annot=True, cmap=cm.viridis)

plt.xlabel('true label')
plt.ylabel('predicted label');


In [ ]:
# let's take a look at our data
for patient_indx in range(0, len(X_scaled)):

    patients_feat = X_scaled[patient_indx,:]
    patients_true_pred = enc.inverse_transform(np.expand_dims(Y[patient_indx,:], 0))[0][0]

    # prediction
    pred = model_tum.predict(np.expand_dims(patients_feat, 0))
    pred = enc.inverse_transform( pred )[0][0]

    print("Patient id: %d \t Predicted: %s \t True Diagnosis: %s\n" 
      %(patient_indx, "Cancer" if pred else "No Cancer", "Cancer" if patients_true_pred else "No Cancer"))


In [ ]:
# let's pic up a specific patient:

# patient ID 0 predicted Cancer. True value Cancer
# patient ID 190 predicted No Cancer, true value Cancer
# patient ID 20 predicted No Cancer. True value No Cancer
# patient ID 291 predicted Cancer. true value No Cancer

patient_indx = 0
patients_feat = X_scaled[patient_indx,:]
patients_true_pred = enc.inverse_transform(np.expand_dims(Y[patient_indx,:], 0))[0][0]

# prediction
pred = model_tum.predict(np.expand_dims(patients_feat, 0))
pred = enc.inverse_transform( pred )[0][0]

print("Patient id: %d \t Predicted: %s \t True Diagnosis: %s\n" 
      %(patient_indx, "Cancer" if pred else "No Cancer", "Cancer" if patients_true_pred else "No Cancer"))


### Why is the algorithm making these predictions?


In [ ]:
#!pip install lime

In [ ]:
import lime
from lime import lime_tabular

# lime also works text data
# lime also works image data

In [ ]:
feature_names = data.columns.to_list()
feature_names.remove(class_var)
feature_names.remove("id")

In [ ]:
# LIME has one explainer for all the models
explainer = lime_tabular.LimeTabularExplainer(X_train, feature_names= feature_names, 
                                                  class_names=["No Cancer", "Cancer"], verbose=False, mode='classification')


#### Explain Patient #0 (True Positive)

In [ ]:
MAX_FEAT = 6

In [ ]:
def predict_proba(input_data):
    return model_tum.predict(input_data)

exp = explainer.explain_instance(patients_feat, predict_proba, num_features=MAX_FEAT)


In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True )

In [ ]:
exp.as_list()

In [ ]:
explanation_plot = exp.as_pyplot_figure()

#### Discussion: Do these results support a True Positive prediction?

**Answer here**

#### Explain Patient #190 (False Positive)


In [ ]:
patient_indx = 190

patients_feat = X_scaled[patient_indx,:]
patients_true_pred = enc.inverse_transform(np.expand_dims(Y[patient_indx,:], 0))[0][0]

# prediction
pred = model_tum.predict(np.expand_dims(patients_feat, 0))
pred = enc.inverse_transform( pred )[0][0]

print("Patient id: %d \t Predicted: %s \t True Diagnosis: %s\n" 
      %(patient_indx, "Cancer" if pred else "No Cancer", "Cancer" if patients_true_pred else "No Cancer"))

model_tum.predict(np.expand_dims(patients_feat, 0)) 

In [ ]:
def predict_proba(input_data):
    return model_tum.predict(input_data)

exp = explainer.explain_instance(patients_feat, predict_proba, num_features=MAX_FEAT)


In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True )

In [ ]:
explanation_plot = exp.as_pyplot_figure()

#### Discussion: Do these results support a False Negative prediction?

**Answer here**

#### Explain Patient #20 (True Negative)

In [ ]:
patient_indx = 20

patients_feat = X_scaled[patient_indx,:]
patients_true_pred = enc.inverse_transform(np.expand_dims(Y[patient_indx,:], 0))[0][0]

# prediction
pred = model_tum.predict(np.expand_dims(patients_feat, 0))
pred = enc.inverse_transform( pred )[0][0]

print("Patient id: %d \t Predicted: %s \t True Diagnosis: %s\n" 
      %(patient_indx, "Cancer" if pred else "No Cancer", "Cancer" if patients_true_pred else "No Cancer"))

model_tum.predict(np.expand_dims(patients_feat, 0)) 

In [ ]:

#exp = explainer.explain_instance(patients_feat, model_tum.predict_proba, num_features=6 )

def predict_proba(input_data):
    return model_tum.predict(input_data)

exp = explainer.explain_instance(patients_feat, predict_proba, num_features=6)


In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True )

In [ ]:
explanation_plot = exp.as_pyplot_figure()

#### Discussion: Do these results support a True Negative prediction?

**Answer here**

#### Explain Patient #541 (False Negative)

In [ ]:
patient_indx = 291

patients_feat = X_scaled[patient_indx,:]
patients_true_pred = enc.inverse_transform(np.expand_dims(Y[patient_indx,:], 0))[0][0]

# prediction
pred = model_tum.predict(np.expand_dims(patients_feat, 0))
pred = enc.inverse_transform( pred )[0][0]

print("Patient id: %d \t Predicted: %s \t True Diagnosis: %s\n" 
      %(patient_indx, "Cancer" if pred else "No Cancer", "Cancer" if patients_true_pred else "No Cancer"))

model_tum.predict(np.expand_dims(patients_feat, 0)) 


In [ ]:
#exp = explainer.explain_instance(patients_feat, model_tum.predict_proba, num_features=15 )

def predict_proba(input_data):
    return model_tum.predict(input_data)

exp = explainer.explain_instance(patients_feat, predict_proba, num_features=15)


In [ ]:
# Show the predictions
exp.show_in_notebook(show_table=True )

In [ ]:
explanation_plot = exp.as_pyplot_figure()

#### Discussion: Do these results support a False Positive prediction?

**Answer here**

## Explaining the Manipulated Criminal Image Dataset

In [ ]:
faces_dataset = pd.read_csv('data/dataset_64_v2.csv', delimiter=',')
faces_dataset

In [ ]:
# total number of images in dataset:
num_images = len(faces_dataset)
num_pixels = len(faces_dataset.columns) - 1

print('Total number of images in the dataset is: ' + str(num_images))
print('Total number of pixels per image is: ' + str(num_pixels))

In [ ]:
# We can also try to understand what is the distribution of criminals / not crimials in our dataset:
criminals = faces_dataset[ faces_dataset['target'] == 1 ]
not_criminals = faces_dataset[ faces_dataset['target'] == 0 ]

print('Criminality distribution:')
print('Number of criminals: ' + str(len(criminals)))
print('Number of not criminals: ' + str(len(not_criminals)))

In [ ]:
# getting some images... 
from skimage.transform import resize

indx = 0
image_0 = faces_dataset.iloc[indx,0:num_pixels]

# image_0 is a vector with 9216 pixels. In order to convert it into an image,
# we need to reshape this vector into a matrix of 96*96 = 9216 pixels

image_0 =  np.array(image_0)
image_0 = np.reshape(image_0, (64, 64))
image_0 = resize(image_0, (128, 128), anti_aliasing=True)

# show image
plt.imshow(image_0,  cmap=plt.cm.gray )

# print target
if( faces_dataset.iloc[indx,num_pixels] == 1):
    print('Image label = CRIMINAL')
else:
    print('Image label = NOT CRIMINAL')

In [ ]:
all_images = []

WIDTH = 64
HEIGHT = 64

for indx in range(0, len(faces_dataset)):
    
    temp_rgb = np.zeros((WIDTH,HEIGHT,3))
    image = faces_dataset.iloc[indx,:].drop(["target"],axis=0).values

    # normalise pixels
    temp = np.reshape(image, (WIDTH, HEIGHT))/255.0

    # add image to each color channel
    temp_rgb[:,:,0] = temp
    temp_rgb[:,:,1] = temp
    temp_rgb[:,:,2] = temp
    
    all_images.append(temp_rgb)

# convert structure to array
X = np.array(all_images)
X.shape

In [ ]:
# separate your dataset: 
# put the variable that you wish to classify (the target) in one variable
# put your features (the pixels) in another variable

y = faces_dataset["target"]


# separate the dataset into test set and train set
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.3,  random_state = 515  )
X_test, X_val, y_test, y_val = train_test_split( X_test, y_test, test_size = 0.5,  random_state = 515 )

print( X_train.shape, X_test.shape, X_val.shape )


In [ ]:
from tensorflow.keras.utils import to_categorical

# convert diagnosis to categorical features with a representation of M x diagnosis matrix
le = LabelEncoder()
le.fit(y)

# convert training, test and validation sets to categorical features
Y_train = np.array(to_categorical(le.transform(y_train)))
Y_test  = np.array(to_categorical(le.transform(y_test)))
Y_val = np.array(to_categorical(le.transform(y_val)))

# check sizes of training set, test set and validation set
print( Y_train.shape, Y_test.shape, Y_val.shape )


In [ ]:
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2], 3)) 

hidden_layer = Conv2D(200, kernel_size=1,activation="relu" )(input_layer)
dropout_layer1 = Dropout(0.2)(hidden_layer)  

hidden_layer2 = Conv2D(100, kernel_size=1,activation="relu" )(dropout_layer1)
dropout_layer2 = Dropout(0.2)(hidden_layer2)  
flatten_layer2 = Flatten()(dropout_layer2)

output_layer = Dense(Y_train.shape[1], activation="softmax", kernel_initializer="glorot_uniform")(flatten_layer2)

# return both the mod and the hidden layer
model_deep = Model(inputs=input_layer, outputs=output_layer)
  
# select optimizer and compile model
#opt = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.003, clipvalue=3)
#opt = Nadam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipvalue=3)
opt = Nadam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipvalue=3)


model_deep.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['accuracy'])


In [ ]:
# visualise the model
model_deep.summary()

In [ ]:
history_deep = model_deep.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=15, verbose=1)


In [ ]:

# evaluate the model
train_loss_deep, train_acc_deep = model_deep.evaluate(X_train, Y_train, verbose=0)
test_loss_deep, test_acc_deep = model_deep.evaluate(X_test, Y_test, verbose=0)
print('Train Accuracy: %.3f, Test Accuracy: %.3f' % (train_acc_deep, test_acc_deep))
print('Train Loss: %.3f, Test Loss: %.3f' % (train_loss_deep, test_loss_deep))

### Explaining Predictions on Image Data

In [ ]:
import lime
from lime.wrappers.scikit_image import SegmentationAlgorithm
from lime import lime_image

explainer = lime_image.LimeImageExplainer()

#### Explaining Predictions

In [ ]:
# Taking a look at the training set
# heavy! avoid running!
#steps = 5
#for indx in range(0, len(X_train), steps ):
#    f, axarr = plt.subplots(1,steps,figsize=(10, 7))
#    
#    plot_range = indx+steps
#    if  plot_range >= len(X_train):
#        plot_range = len(X_train)
#    
#    plt_indx = 0
#    for sub_indx in range(indx, plot_range):
#        image_test = X_train[sub_indx]
#        label = Y_train[sub_indx]

#        groundtruth =  "Criminal" if np.argmax(label) else "Not Criminal"
#        axarr[plt_indx].title.set_text('indx = ' + str(sub_indx) + " " + groundtruth )
#        axarr[plt_indx].imshow(image_test,  cmap=plt.cm.gray )
#        plt_indx = plt_indx+1

import matplotlib.pyplot as plt
import numpy as np

def plot_images(start_index, step_size, X_data, Y_data):
    """Plot step_size images starting from start_index."""
    f, axarr = plt.subplots(1, step_size, figsize=(10, 7))
    
    for i in range(step_size):
        if start_index + i >= len(X_data):
            break
        
        image_test = X_data[start_index + i]
        label = Y_data[start_index + i]

        groundtruth = "Criminal" if np.argmax(label) else "Not Criminal"
        axarr[i].title.set_text('indx = ' + str(start_index + i) + " " + groundtruth)
        axarr[i].imshow(image_test, cmap=plt.cm.gray)

    plt.show()
    plt.close()

# Display images in groups of 5
step_size = 5
for indx in range(0, len(X_train), step_size):
    plot_images(indx, step_size, X_train, Y_train)


In [ ]:
# taking a look at the validation set
steps = 5
for indx in range(0, len(X_val), steps ):
    f, axarr = plt.subplots(1,steps,figsize=(10, 7))
    
    plot_range = indx+steps
    if  plot_range >= len(X_val):
        plot_range = len(X_val)
    
    plt_indx = 0
    for sub_indx in range(indx, plot_range):
        image_test = X_val[sub_indx]
        label = Y_val[sub_indx]

        groundtruth =  "Criminal" if np.argmax(label) else "Not Criminal"
        axarr[plt_indx].title.set_text('indx = ' + str(sub_indx) + " " + groundtruth )
        axarr[plt_indx].imshow(image_test,  cmap=plt.cm.gray )
        plt_indx = plt_indx+1

    plt.show()

In [ ]:
# Taking a look at the test set
steps = 3
for indx in range(0, len(X_test), steps ):
    f, axarr = plt.subplots(1,steps,figsize=(10, 7))
    
    plot_range = indx+steps
    if  plot_range >= len(X_test):
        plot_range = len(X_test)
    
    plt_indx = 0
    for sub_indx in range(indx, plot_range):
        image_test = X_test[sub_indx]
        label = Y_test[sub_indx]

        groundtruth =  "Criminal" if np.argmax(label) else "Not Criminal"
        axarr[plt_indx].title.set_text('indx = ' + str(sub_indx) + " " + groundtruth )
        axarr[plt_indx].imshow(image_test,  cmap=plt.cm.gray )
        plt_indx = plt_indx+1

    plt.show()    

In [ ]:
# Catarina in index 147 in Validation set
# Andrew in index 32 in Training set

indx = 32
data_lookup_X = X_train
data_lookup_Y = Y_train

image_test = data_lookup_X[indx]
label_test = "Criminal" if np.argmax( data_lookup_Y[indx] ) else "Not Criminal" 
print( label_test )
plt.imshow(image_test,  cmap=plt.cm.gray )

In [ ]:
# interprete results
explanation = explainer.explain_instance(image_test, model_deep.predict, top_labels=5, hide_color=1, num_samples=1000)


In [ ]:
from skimage.segmentation import mark_boundaries

f, axarr = plt.subplots(figsize=(4, 6))
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=10, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask), interpolation="nearest", aspect="auto", cmap=plt.cm.gray)


In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)

f, axarr = plt.subplots(1,2,figsize=(10, 7))
axarr[0].imshow(mark_boundaries(temp / 2 + 0.5, mask), interpolation="nearest", aspect="auto")
axarr[0].title.set_text(label_test)
axarr[1].imshow(image_test, interpolation="nearest", aspect="auto")
axarr[1].title.set_text(label_test)
